In [ ]:
from quantumnet.components import Controller, Network
from quantumnet.utils import generate_random_request, register_request
path = "resultados/reativo/sync.csv"
rede = Network()
controlador = Controller(rede)
row, col = 3, 4
rede.set_ready_topology("Grade", row, col)
# Criação de requisições
fmin_range = (0.5, 1)
neprs_range = (2, 4)
num_requests = 1000
requests = list(generate_random_request(len(rede.hosts)-1, fmin_range, neprs_range) for i in range(num_requests))
requestscopy = requests.copy()

In [8]:
# Parte adicional para simulação com timeslot
def simulate_requests_with_timeslots(rede, requests, controlador):
    """
    Simula o processamento de requests com a introdução de timeslots.
    - Demora 1 time-slot se a regra existe.
    - Demora 3 time-slots se uma nova regra precisa ser adicionada.
    """
    current_time = 0  # Tempo global da simulação
    for request in requests:
        print(f"[Time {current_time}]")
        alice = rede.get_host(request.alice)
        rule = alice.find_rule_by_request(request)

        if rule == False:  # Caso não exista um match na tabela
            request.starttime = current_time
            current_time += 3  # Delay de 3 time-slots para registrar nova regra
            controlador.add_match_route_rule_in_host_reactive(request)
            rule = alice.find_rule_by_request(request)
            controlador.run_rule(rule[1])
            request.endtime = current_time
            
            # Registra no CSV como um novo registro
            register_request(request, "Novo Registro", path)
        else:  # Caso já exista a regra
            request.starttime = current_time
            current_time += 1  # Delay de 1 time-slot para execução
            controlador.run_rule(rule[1])
            request.endtime = current_time
            
            # Registra no CSV como já registrado
            register_request(request, "Já Registrado", path)
        
        # Exibir informações da requisição
        print(f"Request {request}: Start Time = {request.starttime}, End Time = {request.endtime}")


In [ ]:
simulate_requests_with_timeslots(rede, requests, controlador)